<h2><center> CSCI - UA 9472 - Artificial Intelligence </center></h2>

<h3><center> Assignment 3: Logical reasoning </center></h3>

<center>Given date: November 8 
</center>
<center><font color='red'>Due date: November 30 </font>
</center>
<center><b>Total: 40 pts </b>
</center>


<center>In this third assignment, we will implement a simple Logical agent by relying on the resolution algorithm of Propositional Logic.</center>

<img src="https://github.com/acosse/Artificial-Intelligence-Fall2021/blob/main/Assignments/Assignment3/simpleVideoGameCave.jpeg?raw=1" width="400" height="300"/>

 = = = = = = = = = = = = = = = = = = = = = \
 **Name:** Haoming(Hammond) Liu \
 **NetID:** hl3797 \
 **Date:** Nov. 29th, 2021 \
  = = = = = = = = = = = = = = = = = = = = =


In [ ]:
import time
import collections
import numpy as np
from copy import deepcopy
from random import choice

import matplotlib.pyplot as plt
from IPython.display import display, HTML
from matplotlib.animation import FuncAnimation

### Introduction: logical propositions

The final objective will be to code our logical agent to succeed in a simple world similar to the Wumpus world discussed in the lectures. The final world we will consider is shown below.

<img src="https://github.com/acosse/Artificial-Intelligence-Fall2021/blob/main/Assignments/Assignment3/MazeTotal.png?raw=1" width="400" height="300"/>

Before designing the full agent, we will focus on a series of simplified environments (see below). In order to help you in your implementation, you are provided with the class 'Expr' and the associated function 'expr' which can be used to store and process logical propositions. The logical expressions are stored as objects consisting of an operator 'op' which can be of the types '&' (and), '|' (or) '==>' (implication) or '<=>' (double implication) as well as '~' (not). A logical expression such as 'A & B' can be stored as a string by means of the function expr() as expr('A & B') or Expr('&', 'A', 'B').

The function expr() takes operator precedence into account so that the two lines

In [ ]:
'''source : AIMA'''

def Symbol(name):
    """A Symbol is just an Expr with no args."""
    return Expr(name)

class Expr:
    """source: Artificial Intelligence: A Modern Approach
    A mathematical expression with an operator and 0 or more arguments.
    op is a str like '+' or 'sin'; args are Expressions.
    Expr('x') or Symbol('x') creates a symbol (a nullary Expr).
    Expr('-', x) creates a unary; Expr('+', x, 1) creates a binary."""

    def __init__(self, op, *args):
        self.op = str(op)
        self.args = args

    # Operator overloads
    def __neg__(self):
        return Expr('-', self)

    def __pos__(self):
        return Expr('+', self)

    def __invert__(self):
        return Expr('~', self)

    def __add__(self, rhs):
        return Expr('+', self, rhs)

    def __sub__(self, rhs):
        return Expr('-', self, rhs)

    def __mul__(self, rhs):
        return Expr('*', self, rhs)

    def __pow__(self, rhs):
        return Expr('**', self, rhs)

    def __mod__(self, rhs):
        return Expr('%', self, rhs)

    def __and__(self, rhs):
        return Expr('&', self, rhs)

    def __xor__(self, rhs):
        return Expr('^', self, rhs)

    def __rshift__(self, rhs):
        return Expr('>>', self, rhs)

    def __lshift__(self, rhs):
        return Expr('<<', self, rhs)

    def __truediv__(self, rhs):
        return Expr('/', self, rhs)

    def __floordiv__(self, rhs):
        return Expr('//', self, rhs)

    def __matmul__(self, rhs):
        return Expr('@', self, rhs)

    def __or__(self, rhs):
        """Allow both P | Q, and P |'==>'| Q."""
        if isinstance(rhs, Expression):
            return Expr('|', self, rhs)
        else:
            return PartialExpr(rhs, self)

    # Reverse operator overloads
    def __radd__(self, lhs):
        return Expr('+', lhs, self)

    def __rsub__(self, lhs):
        return Expr('-', lhs, self)

    def __rmul__(self, lhs):
        return Expr('*', lhs, self)

    def __rdiv__(self, lhs):
        return Expr('/', lhs, self)

    def __rpow__(self, lhs):
        return Expr('**', lhs, self)

    def __rmod__(self, lhs):
        return Expr('%', lhs, self)

    def __rand__(self, lhs):
        return Expr('&', lhs, self)

    def __rxor__(self, lhs):
        return Expr('^', lhs, self)

    def __ror__(self, lhs):
        return Expr('|', lhs, self)

    def __rrshift__(self, lhs):
        return Expr('>>', lhs, self)

    def __rlshift__(self, lhs):
        return Expr('<<', lhs, self)

    def __rtruediv__(self, lhs):
        return Expr('/', lhs, self)

    def __rfloordiv__(self, lhs):
        return Expr('//', lhs, self)

    def __rmatmul__(self, lhs):
        return Expr('@', lhs, self)

    def __call__(self, *args):
        """Call: if 'f' is a Symbol, then f(0) == Expr('f', 0)."""
        if self.args:
            raise ValueError('Can only do a call for a Symbol, not an Expr')
        else:
            return Expr(self.op, *args)

    # Equality and repr
    def __eq__(self, other):
        """x == y' evaluates to True or False; does not build an Expr."""
        return isinstance(other, Expr) and self.op == other.op and self.args == other.args

    def __lt__(self, other):
        return isinstance(other, Expr) and str(self) < str(other)

    def __hash__(self):
        return hash(self.op) ^ hash(self.args)

    def __repr__(self):
        op = self.op
        args = [str(arg) for arg in self.args]
        if op.isidentifier():   # f(x) or f(x, y)
            return '{}({})'.format(op, ', '.join(args)) if args else op
        elif len(args) == 1:    # -x or -(x + 1)
            return op + args[0]
        else:                   # (x - y)
            opp = (' ' + op + ' ')
            return '(' + opp.join(args) + ')'
        
def expr(x):
    """
    Shortcut to create an Expression. x is a str in which:
    - identifiers are automatically defined as Symbols.
    - ==> is treated as an infix |'==>'|, as are <== and <=>.
    If x is already an Expression, it is returned unchanged. Example:
    >>> expr('P & Q ==> Q')
    ((P & Q) ==> Q)
    """
    return eval(expr_handle_infix_ops(x), defaultkeydict(Symbol)) if isinstance(x, str) else x

def expr_handle_infix_ops(x):
    """
    Given a str, return a new str with ==> replaced by |'==>'|, etc.
    >>> expr_handle_infix_ops('P ==> Q')
    "P |'==>'| Q"
    """
    infix_ops = '==> <== <=>'.split()
    for op in infix_ops:
        x = x.replace(op, '|' + repr(op) + '|')
    return x

class defaultkeydict(collections.defaultdict):
    """
    Like defaultdict, but the default_factory is a function of the key.
    >>> d = defaultkeydict(len); d['four']
    4
    """

    def __missing__(self, key):
        self[key] = result = self.default_factory(key)
        return result

    
class PartialExpr:
    """Given 'P |'==>'| Q, first form PartialExpr('==>', P), then combine with Q."""

    def __init__(self, op, lhs):
        self.op, self.lhs = op, lhs

    def __or__(self, rhs):
        return Expr(self.op, self.lhs, rhs)

    def __repr__(self):
        return "PartialExpr('{}', {})".format(self.op, self.lhs)


Number = (int, float, complex)
Expression = (Expr, Number)

#### Utility functions

In [ ]:
OP_BASE = {'&': True, '|': False}

# credits: https://github.com/aimacode/aima-python
def flatten(op, args):
    '''
    flatten clause & fetch components
    >>> flatten('|', expr('A | (B | C)').args)
    [A, B, C]
    '''
    flat_args = []
    def fetch(subargs):
        for arg in subargs:
            if arg.op == op:
                fetch(arg.args)
            else:
                flat_args.append(arg)
    fetch(args)
    return flat_args

# credits: https://github.com/aimacode/aima-python
def merge(op, args):
    '''
    flatten clause & connect args with op
    >>> merge('&', expr('(A & B) & (B | C)').args)
    (A & B & (B | C))
    '''
    # print(op, args)
    args = flatten(op, args)
    # print(args)
    if len(args) == 0:
        return OP_BASE[op]
    elif len(args) == 1:
        return args[0]
    else:
        return Expr(op, *args)

def adjacents(curr_h, curr_w, world_h=5, world_w=5):
    '''get valid adjacent cells at given position'''
    cells = []
    if curr_h-1 >= 0:
        cells.append((curr_h-1, curr_w))
    if curr_h+1 < world_h:
        cells.append((curr_h+1, curr_w))
    if curr_w-1 >= 0:
        cells.append((curr_h, curr_w-1))
    if curr_w+1 < world_w:
        cells.append((curr_h, curr_w+1))
    return cells

def all_adjacents(pos_list, world_h=5, world_w=5):
    '''get valid adjacent cells at multiple position'''
    cells = []
    for h, w in pos_list:
        cells += adjacents(h, w, world_h, world_w)
    return cells

def pairs(clauses):
    '''iter all possible pairs of clauses'''
    val = list(clauses.values())
    for i in range(len(val)):
        for j in range(i+1, len(val)):
            yield (val[i], val[j])

def rules(s1, s2, world_h=5, world_w=5, imp='==>'):
    '''init rules with given symbols, implications, and world size'''
    ans = []
    for h in range(world_h):
        for w in range(world_w):
            for adj in adjacents(h, w):
                c = ' '.join([get_symbol(s1, (h, w)), imp, get_symbol(s2, adj)])
                ans.append(expr(c))
    # print(ans)
    return ans

def display_movie(world, path, delay=1, world_size=(5, 5)):
    '''Display a moview with given world & path'''

    # init colors
    def init_obj_color(key, color_id):
        if key not in world:
            return
        if isinstance(world[key], list):
            # print(key, world[key])
            for pos in world[key]:
                maze[pos] = color_id
        else:
            maze[world[key]] = color_id        

    # plot a new frame
    def animate(frame_idx):
        frame_idx = frame_idx // delay
        plot.set_data(maze_frames[frame_idx])
        return (plot, )

    # init world
    print('\ngenerating movie...\n')
    maze = np.full(world_size, fill_value=8)
    init_obj_color('Shine', 7)
    init_obj_color('Oooo', 7)
    init_obj_color('Six', 7)
    init_obj_color('Ghost', 2)
    init_obj_color('Spike', 2)
    init_obj_color('Treasure', 1)
    init_obj_color('Door', 1)

    # generate frames
    maze_frames = []
    for i in range(len(path)):
        maze[path[i]] = 5
        maze_with_agent = maze.copy()
        maze_with_agent[path[i]] = 3
        maze_frames.append(maze_with_agent)
    
    # init first frame
    fig, ax = plt.subplots()
    plot = ax.matshow(maze_frames[0], cmap=plt.cm.Set2)

    # make animation
    frame_num = len(maze_frames) * delay
    anim = FuncAnimation(fig, animate, frames=frame_num, interval=30, blit=True)
    display(HTML(anim.to_html5_video()))
    fig.delaxes(ax)

# check raw atomic sentence: expr('A') => True
ras_flag = lambda p: (p.op.isalnum()) and (p.args == ())
# check negated atomic sentence: expr('~A') => True
nas_flag = lambda p: (p.op == '~') and (len(p.args) == 1)
# check atomic sentence: [raw a.s.] or [negated a.s.] => True
atm_flag = lambda p: (ras_flag(p)) or (nas_flag(p) and ras_flag(p.args[0]))

# check disjuction of atomic sentences: expr('(A | ~B)') => True
doa_flag = lambda p: (p.op == '|') and all([atm_flag(q) for q in p.args])
# check CNF components: [atomic] or [disjuction of atomic] => True
cmp_flag = lambda p: atm_flag(p) or doa_flag(p)
# check conj. of disj. of atomic sentences: expr('(A | ~B) & C') => True
cod_flag = lambda p: (p.op == '&') and all([cmp_flag(q) for q in p.args])
# check CNF: [CNF comp.] or [conj. of disj.] => True
cnf_flag = lambda p: cmp_flag(p) or cod_flag(p)

# get symbols of CNF components: expr('(A | ~B)') => [A, ~B]
cmp_symb = lambda p: [p] if atm_flag(p) else list(p.args)
# get CNF components: expr('(A | ~B) & C') => [(A | ~B), C]
cnf_comp = lambda p: [p] if cmp_flag(p) else list(p.args)

# get symbol with given prefix & position
get_symbol = lambda s, pos: '%s%d%d' % (s, pos[0], pos[1])
# get a list with a given item omitted
omit_items = lambda seq, item: [x for x in seq if x != item]
# get the symbol and flag of an atomic sentence (e.g. '~A' => 'A', False)
parse_atomic = lambda p: (str(p)[1:], False) if p.op == '~' else (str(p), True)


#### Question 1: to CNF (7pts)

Now that we can create a knowledge base, in order to implement the resolution algorithm that will ultimately enable our agent to leverage the information from the environment, we need our sentences to written in conjunctive normal form (CNF). That requires a number of steps which are recalled below:

- Biconditional elimination: $(\alpha \Leftrightarrow \beta) \equiv ((\alpha\Rightarrow \beta) \wedge (\beta \Rightarrow \alpha))$
- Implication elimination $\alpha \Rightarrow \beta \equiv \lnot \alpha \vee \beta$
- De Morgan's Law $\lnot (A\wedge B) = (\lnot A \vee \lnot B)$, $\lnot(\alpha \vee B) \equiv (\lnot A \wedge \lnot B)$
- Distributivity of $\vee$ over $\wedge$: $(\alpha \vee (\beta \wedge \gamma)) \equiv ((\alpha \vee \beta) \wedge (\alpha \vee \gamma))$

Relying on the function propositional_exp given above (to avoid having all the questions depend on question 1, we will now rely on this implementation from AIMA), complete the function below which should return the CNF of the logical propostion $p$.  

In [ ]:
def eliminate_imp(p):
    '''
    implication elimination (i.e. "<=>", "<==", "==>")
    Note: single/double cases have been merged for efficiency
    >>> eliminate_imp(expr('A <=> B'))
    ((~A | B) & (~B | A))
    '''
    # case 1: atomic sentence
    if atm_flag(p):
        return p
    args = tuple(map(eliminate_imp, p.args))
    e1, e2 = args[0], args[-1]
    # case 2: implications
    if p.op == '==>':
        return (~e1 | e2)
    elif p.op == '<==':
        return (~e2 | e1)
    elif p.op == '<=>':
        return (~e1 | e2) & (~e2 | e1)
    # case 3: otherwise
    else:
        return Expr(p.op, *args)

def move_neg(p):
    '''
    move '~' inside: De Morgan's Law & neg elimination
    >>> move_neg(expr('~(A & ~B)'))
    (~A | B)
    '''
    # case 1: atomic sentence
    if atm_flag(p):
        return p
    # case 2: with '~' to move
    elif p.op == '~':
        e = p.args[0]
        dml = lambda x: move_neg(~x)
        if e.op == '~':
            # ~~A => A
            return move_neg(e.args[0])
        else:
            # ~(A & B) => (~A | ~B)
            # ~(A | B) => (~A & ~B)
            new_op = ('&' if e.op == '|' else '|')
            return merge(new_op, tuple(map(dml, e.args)))
    # case 3: otherwise
    else:
        return Expr(p.op, *tuple(map(move_neg, p.args)))

def distribute(p):
    '''
    distributivity of ∨ over ∧
    >>> distribute(expr('(A & B) | C'))
    ((A | C) & (B | C))
    '''

    def find_first_op(args, op):
        for arg in args:
            if arg.op == op:
                return arg
        return None

    # case 1: atomic sentence
    if atm_flag(p):
        return p

    # case 2a: there might be '&' inside
    if p.op == '|':
        p = merge('|', p.args)
        if len(p.args) == 1:
            return distribute(p.args[0])
        conj = find_first_op(p.args, '&')
        if conj is None:
            return merge(p.op, p.args)
        others = [arg for arg in p.args if arg is not conj]
        others = (others[0] if len(others) == 1 else merge('|', others))
        clause = [(e|others) for e in conj.args]
        return merge('&', tuple(map(distribute, clause)))
    # case 2b: there might be '&' inside (easier than '|')
    elif p.op == '&':
        return merge('&', tuple(map(distribute, p.args)))
    # otherwise
    else:
        return p

def beautify_cmp(p):
    '''
    sort symbol & remove all "A | ~A" in the components
    >>> beautify_cmp(expr('A | ~A | B | C'))
    (B | C)
    '''
    if atm_flag(p):
        return p
    remove_set = set()
    symbol_set = set(cmp_symb(p))
    # print(symbol_set)
    for s in symbol_set:
        if ~s in symbol_set:
            remove_set = remove_set | {s, ~s}
    return merge('|', tuple(sorted(list(symbol_set - remove_set))))

def beautify_cnf(p):
    '''beautify all the components inside a CNF'''
    if atm_flag(p):
        return p
    new = tuple(sorted(list(set([beautify_cmp(c) for c in cnf_comp(p)]))))
    if (len(new) == 0) or (False in new):
        return False
    elif len(new) == 1:
        return new[0]
    else:
        return merge('&', new)

def to_CNF(p):
    '''return the CNF of proposition p'''    
    if not isinstance(p, Expr):
        p = expr(p)
    return beautify_cnf(distribute(move_neg(eliminate_imp(p))))

e = expr('A <=> B')
to_CNF(e)

((A | ~B) & (B | ~A))

#### Question 2: The resolution rule (7pts)

Now that you have a function that can turn any logical sentence to a CNF, we will code the resolution rule. For any two propositions $p_1$ and $p_2$ written in conjunctive normal form, write a function Resolution that returns empty if the resolution rule applied to the two sentences cannot produce any new sentence and that returns the set of all propositions $p_i$ following from the resolution of $p_1$ and $p_2$ otherwise.  

Study the disjuncts of both $p_1$ and $p_2$. For each of the disjunct in $p_1$ try to find in $p_2$ the negation of that disjunct. If this negation appears, returns the proposition resulting from combining $p_1$ and $p_2$ with a disjunction. 

In [ ]:
'''
Note: Originally, I implemented this function by the pseudocode in textbook,
which works quite slow. Inspired by Haiyang, I implement the resolution rule
based on symbols and set operations, which works amazingly fast.
This brilliant idea credits to Haiyang.
'''

def resolution_rule(p1, p2): 
    '''
    applies the resolution rule on two propositions p1 and p2
    return -1 for invalid cases (e.g. empty clause, insolvable, etc.)
    otherwise return the resolution result
    '''

    def split(p_set):
        pos_set = {str(s) for s in p_set if s.op !='~'}
        neg_set = {str(s)[1:] for s in p_set if s.op =='~'}
        return pos_set, neg_set
    
    def combine(pos_set, neg_set):
        pos_lst = [expr(s) for s in sorted(list(pos_set))]
        neg_lst = [expr('~' + s) for s in sorted(list(neg_set))]
        return pos_lst + neg_lst

    # get all symbols & split to pos/neg
    p1_set, p2_set = set(cmp_symb(p1)), set(cmp_symb(p2))
    p1_pos, p1_neg = split(p1_set)
    p2_pos, p2_neg = split(p2_set)
    # print(p1_set, p2_set)
    # print(p1_pos, p1_neg)
    # print(p2_pos, p2_neg)
    if len(p1_pos & p2_neg) == 0 and len(p1_neg & p2_pos) == 0:
        return -1

    # fast resolution on atomic sentences (credits Haiyang)
    pos = (p1_pos - p2_neg) | (p2_pos - p1_neg)
    neg = (p1_neg - p2_pos) | (p2_neg - p1_pos)
    # print(pos, neg)
    if len(pos & neg):
        return -1
    
    args = combine(pos, neg)
    if len(args) == 0:
        return Expr('()')
    elif len(args) == 1:
        return Expr(args[0])
    else:
        return Expr("|", *args)

# e1 = expr('A')
# e2 = expr('~A')
e1 = expr('A | B')
e2 = expr('~A | C')
resolution_rule(e1, e2)

(B | C)

#### Question 3: The resolution algorithm (6pts)

Now that we have a resolution function we can embed it into a resolution algorithm. Complete the function Resolution below which implements the resolution algorithm. The algorithm takes as input a knowledge base written in conjunctive normal form, and a proposition $\alpha$ and should return true or false (as stored in the variable 'is_entailed') depending on whether $\alpha$ is entailed by the knowledge base KB.

In [ ]:
# Note: The resolution algorithm has been integrated in the KnowledgeBase class

#### Question 4 (8pts): A first logical agent

Given our resolution algorithm, we will finally be able to design our first logical agent. As a first step, we consider a simple agent located on the bottom left cell of a 5 by 5 grid world. The world contains a single threat represented by a ghost which occupies a single cell and emits a loud noise ('OOooo') audible in the immediately adjacent cells. 

To implement the agent, we will use the following approach:

Each cell will be represented by its (x,y) coordinate ((0,0) being the bottom leftmost cell). On top of this we will consider the following symbols 

- $D_{(x, y)}$ (which you can store as the string 'Dxy' or 'D_xy' as you want) indicating whether there is an exit on the cell or not (when the agent reach the exit, the simulation ends), 

- $G_{(x, y)}$ which encodes whether the ghost is located on cell $(x, y)$ or not

- $O_{(x, y)}$ which encodes whether a "Ooooo" is heard on the cell $(x, y)$

Using those three symbols, the state of the world can be defined with a total of 3*25 symbols. 

In a while loop running until the door is found, code a simple logical agent that moves at random in the non-threatening cells until it finds the escape cell. Also consider the following:

- The agent should keep track of a knowledge base KB (list of logically true propositions together with a dictionnary storing the symbols that are known to be true) including all the sentences that the agent can generate based on the information it collected at the previous steps. 


- You might want to have a simple function, which given a symbol returns the adjacent symbols (symbols corresponding to spatially adjacent cells). you can store those as strings


- The agent should be equipped with the resolution algorithm that you coded above. Before each new move, the next cell should be determined at random (from the set of all non visited cells) and the agent should use the resolution algorithm to determine whether the ghost is on the cell or not. If there is no indication that the ghost is located on the cell, the agent should make the move. 

<img src="https://github.com/acosse/Artificial-Intelligence-Fall2021/blob/main/Assignments/Assignment3/MazeGhostb.png?raw=1" width="400" height="300"/>

In [ ]:
class KnowledgeBase(object):

    def __init__(self, *clauses):
        self.symbols = {}   # store T/F of some specific symbols
        self.clauses = {}   # store all clauses in KB
        if len(clauses) > 0:
            self.tell(clauses[0] if len(clauses) == 1 else Expr("&", *clauses))
    
    # add a sentence into KB
    def tell(self, sentence, to_cnf=False):
        # convert to CNF as needed
        if not cnf_flag(sentence):
            sentence = to_CNF(sentence)
            # print('cnf', sentence)
        # omit useless sentence
        if sentence == False:
            return
        # add each CNF component
        for c in cnf_comp(sentence):
            self.tell_comp(c)
    
    # add a CNF component into KB
    def tell_comp(self, p):
        # print('learn:', p)
        # quick way to learn an atomic sentence
        if atm_flag(p):

            # parse p & backup the knowledge in self.symbols
            s, f = parse_atomic(p)
            if (s in self.symbols) and (self.symbols.get(s, False) != f):
                raise ValueError('Contradiction: symbol [%s].' % s)
            self.symbols[s] = f

            # pop all useless clauses (e.g. 'A' is true => pop (A | B))
            clauses_same = self.find_clauses(str(p))
            # print('clauses_same:', clauses_same)
            for c in clauses_same:
                self.clauses.pop(c)
                # print('pop:', self.clauses.pop(c))
            
            # update related clauses (e.g. 'A' is true => change (~A | B) to B)
            neg_p = move_neg(~p)
            clauses_diff = self.find_clauses(str(neg_p))
            # print('clauses_diff:', clauses_diff)
            for c in clauses_diff:
                new_args = tuple(omit_items(self.clauses[c].args, neg_p))
                if len(new_args) == 1:
                    self.tell_comp(new_args[0])
                    # print('new finding:', new_args[0])
                else:
                    self.clauses[c] = merge('|', new_args)
                    # print('updated:', self.clauses[c])
        
        # add p to KB clauses
        self.clauses[str(p)] = p
    
    # find all existing clauses with given symbol
    def find_clauses(self, p):
        return [c for c in self.clauses if c.find(p) != -1]

    # query the entailment for some alpha
    def ask(self, alpha):
        # skip resolution if we already have the answer in self.symbols
        if atm_flag(alpha):
            s, f = parse_atomic(alpha)
            # print(self.symbols)
            if s in self.symbols:
                return self.symbols[s] == f
        # check entailment
        ans = self.resolution(alpha)
        # learn new knowledge as needed
        if ans:
            self.tell(alpha)
        return ans
    
    # apply resolution algorithm to check entailment
    def resolution(self, alpha):
        clauses = deepcopy(self.clauses)
        query = to_CNF(Expr('~', alpha))
        if isinstance(query, bool):
            return False
        clauses.update({str(i): i for i in cnf_comp(query)})
        while True:
            new_clauses = {}
            # try to resolve all possible pairs
            for c1, c2 in pairs(clauses):
                resolvent = resolution_rule(c1, c2)
                if resolvent == -1:
                    continue
                if resolvent == Expr('()'):
                    return True
                if str(resolvent) not in clauses:
                    # print('res:', resolvent)
                    clauses[str(resolvent)] = resolvent
                    new_clauses[str(resolvent)] = resolvent
            
            # [keep looping] or [stop here with an answer 'no']
            if len(new_clauses):
                continue
            else:
                return False          

    def __len__(self):
        return len(self.clauses)

    def __repr__(self):
        return str(self.clauses.keys())
    
    def __str__(self):
        return str(self.clauses.keys())


In [ ]:

# set info_flag to True: print out all steps
info_flag = False

world = {
    'Door': (4, 4),
    'Ghost': [(2, 1)],
    'Oooo': all_adjacents([(2, 1)])
}

KB = KnowledgeBase(*rules('Ghost', 'Oooo'))

step = 0
path = []
max_iter = 1000
exit_door = False
agent_h, agent_w = 0, 0
start_time = time.time()

while (not exit_door) and (step < max_iter):
    
    '''Agent should explore the world at random, probing the 
    knowledge base for any potential threat. if the KB does not 
    indicate any specific threat, the agent should move in one of 
    the adacent (cleared) cell. The simulation ends when the agent 
    reaches the exit door'''

    step += 1
    curr_pos = agent_h, agent_w
    path.append(curr_pos)

    # status check
    if curr_pos == world['Door']:
        print('\nFound exit at %s. You WIN!' % str(world['Door']))
        break
    if curr_pos in world['Ghost']:
        print('\nYou are killed by the Ghost!')
        break

    # learning
    info = 'Step to (%d, %d);' % (agent_h, agent_w)
    KB.tell_comp(expr(get_symbol('~Ghost', curr_pos)))
    if curr_pos in world['Oooo']:
        info = 'Heard Oooo at (%d, %d);' % (agent_h, agent_w)
        KB.tell_comp(expr(get_symbol('Oooo', curr_pos)))
    else:
        KB.tell_comp(expr(get_symbol('~Oooo', curr_pos)))    
    # print('KB size:', len(KB))
    # print(KB)

    next_cells = adjacents(agent_h, agent_w)
    while True:
        cell = choice(next_cells)
        query = expr(get_symbol('~Ghost', cell))
        # print('query:', query)
        if KB.ask(query):
            break
        next_cells.remove(cell)
        if len(next_cells) == 0:
            print('\nNowhere to go!')
            exit_door = True
            break
    agent_h, agent_w = cell
    if info_flag:
        print(info.ljust(22), end='\n' if step % 5 == 0 else ' ')

if step == max_iter:
    print('\nMax iter num is reached!')

print('\ntime: %.2fs' % (time.time() - start_time))
print('path:', path)
print('path length:', len(path))
print('KB size:', len(KB))


Found exit at (4, 4). You WIN!

time: 0.01s
path: [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (0, 4), (0, 3), (0, 4), (1, 4), (1, 3), (0, 3), (0, 4), (1, 4), (1, 3), (1, 4), (0, 4), (1, 4), (2, 4), (1, 4), (1, 3), (1, 4), (0, 4), (0, 3), (1, 3), (0, 3), (1, 3), (1, 4), (2, 4), (1, 4), (1, 3), (2, 3), (3, 3), (2, 3), (2, 4), (1, 4), (1, 3), (2, 3), (3, 3), (2, 3), (2, 4), (2, 3), (2, 4), (1, 4), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
path length: 51
KB size: 50


In [ ]:
# set movie_flag to True: generate a movie
movie_flag = False

if movie_flag:
    display_movie(world, path, delay=2)

#### Question 5: Getting used to danger.. (6pts)

Now that our agent knows how to handle a ghost, we will increase the level of risk by including spike traps in the environment. 

- Our agent has an edge: evolution has endowed it with a sort of additional ($6^{th}$) sense so that it can feel something 'bad' is about to happen when it is in a cell adjacent to a trap. We represent this ability with the eye combined with the exclamation mark. 


- Since, as we all know, ghosts are purely imaginary entities, the $6th$ sense only works for the spike traps.  


- The ghost can still be located by means of the noise it generates which can be heard on all adjacent cells.

<img src="https://github.com/acosse/Artificial-Intelligence-Fall2021/blob/main/Assignments/Assignment3/MazeTrapb.png?raw=1" width="400" height="300"/>

Starting from the agent you designed in the previous questions, improve this agent so that it takes into account the spike traps. You should now have a knowledge base defined on a total of 25*5 symbols describing whether each cell contains a ghost, a 'OOoo' noise, a spike trap, activated the agent's'6th sense', or contains the exit door. The search ends when the agent reaches the door.   

In [ ]:

# set info_flag to True: print out all steps
info_flag = False

'''
The given world is not escapable regardless of max_iter, so I comment it below.
Instead, Another escapable world setting is used for testing here.
'''

# world = {
#     'Door': (4, 4),
#     'Ghost': [(2, 1)],
#     'Oooo': all_adjacents([(2, 1)]),
#     'Spike': [(1, 3), (3, 4)],
#     'Six': all_adjacents([(1, 3), (3, 4)]),
# }

world = {
    'Door': (4, 4),
    'Ghost': [(2, 1)],
    'Oooo': all_adjacents([(2, 1)]),
    'Spike': [(0, 4), (3, 4)],
    'Six': all_adjacents([(0, 4), (3, 4)]),
}

KB = KnowledgeBase(*(rules('Ghost', 'Oooo') + rules('Spike', 'Six')))

step = 0
path = []
max_iter = 1000
exit_door = False
agent_h, agent_w = 0, 0
start_time = time.time()

while (not exit_door) and (step < max_iter):
    
    '''Complete the loop with the simulation of the ghost 
    + spike trap logical agent. The simulation should start 
    from the bottom leftmost cell'''

    step += 1
    curr_pos = agent_h, agent_w
    path.append(curr_pos)

    # status check
    if curr_pos == world['Door']:
        print('\nFound exit at %s. You WIN!' % str(world['Door']))
        break
    if curr_pos in world['Ghost']:
        print('\nYou are killed by the Ghost!')
        break
    if curr_pos in world['Spike']:
        print('\nYou are killed by the Spike!')
        break

    # learning
    info = 'Step to (%d, %d);' % (agent_h, agent_w)
    KB.tell_comp(expr(get_symbol('~Ghost', curr_pos)))
    KB.tell_comp(expr(get_symbol('~Spike', curr_pos)))
    if curr_pos in world['Oooo']:
        info = 'Heard Oooo at (%d, %d);' % (agent_h, agent_w)
        KB.tell_comp(expr(get_symbol('Oooo', curr_pos)))
    else:
        KB.tell_comp(expr(get_symbol('~Oooo', curr_pos)))
    if curr_pos in world['Six']:
        info = 'Sixth sense at (%d, %d);' % (agent_h, agent_w)
        KB.tell_comp(expr(get_symbol('Six', curr_pos)))
    else:
        KB.tell_comp(expr(get_symbol('~Six', curr_pos)))
    # print('KB size:', len(KB))
    # print(KB)

    next_cells = adjacents(agent_h, agent_w)
    while True:
        cell = choice(next_cells)
        query1 = expr(get_symbol('~Ghost', cell))
        query2 = expr(get_symbol('~Spike', cell))
        # print('query:', query1, query2)
        if KB.ask(query1) and KB.ask(query2):
            break
        next_cells.remove(cell)
        if len(next_cells) == 0:
            print('\nNowhere to go!')
            exit_door = True
            break
    agent_h, agent_w = cell
    if info_flag:
        print(info.ljust(24), end='\n' if step % 5 == 0 else ' ')

if step == max_iter:
    print('\nMax iter num is reached!')

print('\ntime: %.2fs' % (time.time() - start_time))
print('path:', path)
print('path length:', len(path))
print('KB size:', len(KB))


Found exit at (4, 4). You WIN!

time: 3.61s
path: [(0, 0), (1, 0), (2, 0), (1, 0), (0, 0), (1, 0), (1, 1), (1, 0), (2, 0), (1, 0), (2, 0), (1, 0), (0, 0), (1, 0), (0, 0), (0, 1), (0, 0), (1, 0), (2, 0), (1, 0), (1, 1), (1, 0), (1, 1), (1, 0), (1, 1), (1, 0), (0, 0), (1, 0), (0, 0), (1, 0), (1, 1), (0, 1), (1, 1), (1, 0), (0, 0), (1, 0), (0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (1, 2), (1, 1), (1, 0), (2, 0), (1, 0), (2, 0), (1, 0), (1, 1), (1, 2), (0, 2), (0, 1), (0, 0), (0, 1), (0, 0), (0, 1), (1, 1), (0, 1), (0, 0), (0, 1), (0, 2), (1, 2), (1, 1), (1, 2), (1, 3), (2, 3), (2, 4), (2, 3), (1, 3), (1, 2), (1, 1), (1, 2), (1, 3), (1, 2), (1, 3), (2, 3), (1, 3), (1, 4), (2, 4), (1, 4), (2, 4), (1, 4), (1, 3), (2, 3), (3, 3), (2, 3), (2, 4), (1, 4), (1, 3), (0, 3), (0, 2), (1, 2), (1, 3), (0, 3), (1, 3), (1, 4), (2, 4), (2, 3), (1, 3), (0, 3), (0, 2), (0, 1), (0, 2), (0, 1), (0, 2), (1, 2), (1, 1), (0, 1), (1, 1), (1, 2), (1, 3), (1, 4), (1, 3), (1, 2), (2, 2), (3, 2), (4, 2), (4, 1), (4, 

In [ ]:
# set movie_flag to True: generate a movie
movie_flag = False

if movie_flag:
    display_movie(world, path, delay=1)

#### Bonus: For where your treasure is..  (6pts)

We finally consider the whole environment. This environment is composed of all the elements from the previous questions but it now also includes a treasure chest. The final objective this time is to find the chest first and then reach the exit. Although some of the previous symbols are omitted for clarity, the ghost can always be located by means of the sound it produces, the agent can still trust its $6^{\text{th}}$ sense regarding the spike trap and the treasure chest can be perceived in adjacent cells, by means of the shine it produces.

When the knowledge base does not indicate any threat, the agent should move at random in one of the adjacent cells. 

The world now contains a total of 25*7 symbols. 


<img src="https://github.com/acosse/Artificial-Intelligence-Fall2021/blob/main/Assignments/Assignment3/fullWorldChest.png?raw=1" width="400" height="300"/>

In [ ]:

# set info_flag to True: print out all steps
info_flag = True

'''
The given world is not escapable regardless of max_iter, so I comment it below.
Instead, Another escapable world setting is used for testing here.
'''

# world = {
#     'Door': (4, 4),
#     'Treasure': (3, 2),
#     'Shine': adjacents(3, 2),
#     'Ghost': [(2, 1)],
#     'Oooo': all_adjacents([(2, 1)]),
#     'Spike': [(1, 2), (2, 0), (3, 4)],
#     'Six': all_adjacents([(1, 2), (2, 0), (3, 4)]),
# }

world = {
    'Door': (4, 4),
    'Treasure': (3, 2),
    'Shine': adjacents(3, 2),
    'Ghost': [(2, 1)],
    'Oooo': all_adjacents([(2, 1)]),
    'Spike': [(2, 0), (3, 4)],
    'Six': all_adjacents([(2, 0), (3, 4)]),
}

KB = KnowledgeBase(*(rules('Ghost', 'Oooo') + rules('Spike', 'Six') + \
                     rules('Treasure', 'Shine')))

step = 0
path = []
max_iter = 1000
found_exit = False
found_treasure = False
agent_h, agent_w = 0, 0
start_time = time.time()

# (not (found_exit and found_treasure))
while step < max_iter:
    
    '''Simulation should complete when the treasure chest 
    has been found and the exit has been reached. 
    The agent should start from bottom leftmost cell'''

    step += 1
    curr_pos = agent_h, agent_w
    path.append(curr_pos)

    # status check
    if (curr_pos == world['Door']) and (not found_exit):
        print('\nFound exit at %s.\n' % str(world['Door']))
        found_exit = True
    if (curr_pos == world['Treasure']) and (not found_treasure):
        print('\nFound treasure at %s.\n' % str(world['Treasure']))
        found_treasure = True
    if found_exit and found_treasure:
        break
    if curr_pos in world['Ghost']:
        print('\nYou are killed by the Ghost!')
        break
    if curr_pos in world['Spike']:
        print('\nYou are killed by the Spike!')
        break

    # learning
    info = 'Step to (%d, %d);' % (agent_h, agent_w)
    KB.tell_comp(expr(get_symbol('~Ghost', curr_pos)))
    KB.tell_comp(expr(get_symbol('~Spike', curr_pos)))
    if curr_pos in world['Oooo']:
        info = 'Heard Oooo at (%d, %d);' % (agent_h, agent_w)
        KB.tell_comp(expr(get_symbol('Oooo', curr_pos)))
    else:
        KB.tell_comp(expr(get_symbol('~Oooo', curr_pos)))
    if curr_pos in world['Six']:
        info = 'Sixth sense at (%d, %d);' % (agent_h, agent_w)
        KB.tell_comp(expr(get_symbol('Six', curr_pos)))
    else:
        KB.tell_comp(expr(get_symbol('~Six', curr_pos)))
    if curr_pos in world['Shine']:
        info = 'Shine at (%d, %d);' % (agent_h, agent_w)
        KB.tell_comp(expr(get_symbol('Shine', curr_pos)))
    else:
        KB.tell_comp(expr(get_symbol('~Shine', curr_pos)))
    # print('KB size:', len(KB))
    # print(KB)

    next_cells = adjacents(agent_h, agent_w)
    while True:
        cell = choice(next_cells)
        query1 = expr(get_symbol('~Ghost', cell))
        query2 = expr(get_symbol('~Spike', cell))
        # print('query:', query1, query2)
        if KB.ask(query1) and KB.ask(query2):
            break
        next_cells.remove(cell)
        if len(next_cells) == 0:
            print('\nNowhere to go!')
            exit_door = True
            break
    agent_h, agent_w = cell
    if info_flag:
        print(info.ljust(24), end='\n' if step % 5 == 0 else ' ')

if step == max_iter:
    print('Max iter num is reached!')
else:
    print('You WIN!')

print('\ntime: %.2fs' % (time.time() - start_time))
print('path:', path)
print('path length:', len(path))
print('KB size:', len(KB))

Step to (0, 0);          Sixth sense at (1, 0);   Step to (0, 0);          Sixth sense at (1, 0);   Step to (0, 0);         
Sixth sense at (1, 0);   Step to (0, 0);          Step to (0, 1);          Step to (0, 0);          Step to (0, 1);         
Heard Oooo at (1, 1);    Sixth sense at (1, 0);   Heard Oooo at (1, 1);    Step to (0, 1);          Heard Oooo at (1, 1);   
Step to (0, 1);          Heard Oooo at (1, 1);    Sixth sense at (1, 0);   Step to (0, 0);          Sixth sense at (1, 0);  
Step to (0, 0);          Sixth sense at (1, 0);   Heard Oooo at (1, 1);    Step to (0, 1);          Step to (0, 2);         
Step to (1, 2);          Shine at (2, 2);         Step to (1, 2);          Step to (0, 2);          Step to (0, 1);         
Step to (0, 2);          Step to (0, 1);          Heard Oooo at (1, 1);    Sixth sense at (1, 0);   Step to (0, 0);         
Step to (0, 1);          Step to (0, 2);          Step to (0, 1);          Heard Oooo at (1, 1);    Step to (0, 1);         


In [ ]:
# set movie_flag to True: generate a movie
movie_flag = True

if movie_flag:
    display_movie(world, path, delay=2)


generating movie...



<Figure size 432x288 with 0 Axes>

#### **Final Thought**
To speed up the agent, we can use a list/set to store visited positions and prioritize unvisited cells. Considering the fact that the questions require the agent to move randomly, I didn't implement that part, but it should work anyway.

#### Wish you have a great day! :-)